In [ ]:
import xarray as xr
import param

import holoviews as hv
colormaps = hv.plotting.list_cmaps()

import hvplot.xarray
from holoviews.selection import link_selections

import panel as pn

import io
import os

In [ ]:
class ValueChanger(param.Parameterized):
    
    calculation_type = pn.widgets.RadioButtonGroup(options=['Absolute', 'Relatif', 'Percentage'], align='end')
    save = pn.widgets.FileDownload(label='Save', align='end', button_type='success')
    apply = pn.widgets.Button(name='Apply', align='end', button_type='primary')
    spinner = pn.widgets.Spinner(name='Replacement Value', value=0, align='end')
    mask = pn.widgets.Checkbox(name='Mask', max_width=100, align='end')
    mask_value = pn.widgets.Spinner(name='Mask Value', value=0)
    attribute = param.String()
    file = param.Parameter()
    colormap = param.String()
    
    ds = param.Parameter()
    selection = link_selections.instance(unselected_alpha=0.4)
    
    file_pane = pn.Row()
    graph_pane = pn.Column()
    options_pane = pn.Column()
    
    def __init__(self, **params):
        self.param.file.default = pn.widgets.FileInput()
        self.param.ds.default = xr.Dataset()
        self.param.attribute.default = pn.widgets.Select(max_width=200, align='end')
        self.param.colormap.default = pn.widgets.Select(options=colormaps, max_width=200, align='end')
        super().__init__(**params)
        self.apply.on_click(self._apply_values)
        self.save.callback = self._save
        self.file_pane.append(self.file)
        self.curvilinear_coordinates = None
        
    @pn.depends("file.value", watch=True)
    def _parse_file_input(self):
        value = self.file.value
        # We are dealing with a h5netcdf file ->
        # The reader can't read bytes so we need to write it to a file like object
        if value.startswith(b"\211HDF\r\n\032\n"):
            value = io.BytesIO(value)
        ds = xr.open_dataset(value)
        self.attribute.options = list(ds.keys())
        self.curvilinear_coordinates = None
        
        number_coordinates_in_system = len(list(ds.coords.variables.values())[0].dims)
        # Standard Grid
        if number_coordinates_in_system == 1:
            pass
        # Curvilinear coordinates
        elif number_coordinates_in_system == 2:
            dims = list(ds[list(ds.coords)[0]].dims)
            # Store the true coordinates for export
            self.curvilinear_coordinates = list(ds.coords)
            # Add the dimension into the coordinates this results in an ij indexing
            ds.coords[dims[0]] = ds[dims[0]]
            ds.coords[dims[1]] = ds[dims[1]]
            # Remove the curvilinear coordinates from the original coordinates
            ds = ds.reset_coords()
        else:
            raise ValueError("Unknown number of Coordinates")
        self.ds = ds
        return True
        
    def _set_values(self):
        hvds = hv.Dataset(self.ds.to_dataframe(dim_order=[*list(self.ds[self.attribute.value].dims)]).reset_index())
        if self.calculation_type.value == 'Absolute':
            hvds.data[self.attribute.value].loc[hvds.select(self.selection.selection_expr).data.index] = self.spinner.value
        elif self.calculation_type.value == 'Relatif':
            hvds.data[self.attribute.value].loc[hvds.select(self.selection.selection_expr).data.index] += self.spinner.value
        elif self.calculation_type.value == 'Percentage':
            hvds.data[self.attribute.value].loc[hvds.select(self.selection.selection_expr).data.index] *=  (100 + self.spinner.value) / 100.
        self.ds[self.attribute.value] = list(self.ds[self.attribute.value].dims), hvds.data[self.attribute.value].values.reshape(*self.ds[self.attribute.value].shape)
        
    def _save(self):
        filename, extension = os.path.splitext(self.file.filename) 
        self.save.filename = filename + "_netcdf-editor" + extension
        ds = self.ds
        # We need to remove the dimension coordinates and reset the curvilinear coordinates
        if self.curvilinear_coordinates is not None:
            ds = self.ds.drop([*self.ds.dims]).set_coords([*self.curvilinear_coordinates])
        return io.BytesIO(ds.to_netcdf())
    
    def _apply_values(self, event):
        self._set_values()
        self.selection.selection_expr = None
        
    def _get_ordered_coordinate_dimension_names(self):
        dimension_names = list(self.ds.coords)
        if 'lat' in dimension_names[0].lower() and 'lon' in dimension_names[1].lower():
            dimension_names = dimension_names[::-1]
        elif 'x' == dimension_names[1].lower() or 'y' == dimension_names[0].lower():
            dimension_names = dimension_names[::-1]
        return dimension_names
        
    @pn.depends("file.filename", watch=True)
    def _toggle_save(self):
        if self.file.filename and len(self.file_pane) == 1:
            self.file_pane.append(self.save)
        elif not self.file.filename and len(self.file_pane) == 2:
            self.file_pane.pop(1)

        
    @pn.depends("file.filename", watch=True)
    def _toggle_options_pane(self):
        self.options_pane.clear()
        if self.file.filename is not None:
            self.options_pane.extend([
                pn.Row(self.attribute, self.colormap, self.mask, self.mask_value),
                pn.Row(self.calculation_type, self.spinner, self.apply), 
            ])
      
    @pn.depends('ds', 'colormap.value', 'mask.value', 'mask_value.value', watch=True)
    def get_plots(self):
        image = self.ds[self.attribute.value].hvplot(*self._get_ordered_coordinate_dimension_names()).opts(cmap=self.colormap.value)
        if self.mask.value:
            range_dict = {}
            range_dict[self.attribute.value] = (self.mask_value.value, self.mask_value.value)
            image = image.opts(clipping_colors = {'min': 'grey', 'max': 'black'}).redim.range(**range_dict)
        self.graph_pane.clear()
        self.graph_pane.append(
            self.selection(image + self.ds[self.attribute.value].hvplot.hist()).opts(hv.opts.Image(tools=['hover']), hv.opts.Histogram(tools=['hover']))
        )
    
    def plot(self):
        return pn.Column(
            self.file_pane,
            self.options_pane,
            self.graph_pane
        )

In [ ]:
vc = ValueChanger()
vc.plot().servable('NetCDF Editor')